In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

# Load the C++ program-level subset
dataset = load_dataset("codeparrot/xlcost-text-to-code", "C++-program-level")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.32k [00:00<?, ?B/s]

xlcost-text-to-code.py:   0%|          | 0.00/7.61k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/4.39M [00:00<?, ?B/s]

C++-program-level/test/0000.parquet:   0%|          | 0.00/379k [00:00<?, ?B/s]

(…)++-program-level/validation/0000.parquet:   0%|          | 0.00/210k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9797 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/909 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/492 [00:00<?, ? examples/s]

In [ ]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
import re

def contains_minimum(example):
    """
    Check if the word 'minimum' appears in the text or code column,
    including as part of compound words like 'minimumValue' or 'minimum_cost'.
    """
    # Define a regular expression to match 'minimum' in any case or as part of a compound word
    pattern = r'minimum'

    # Search in both 'text' and 'code' columns
    return (
        re.search(pattern, example['text'], re.IGNORECASE) is not None or
        re.search(pattern, example['code'], re.IGNORECASE) is not None
    )

In [ ]:
filtered_train_dataset = train_dataset.filter(contains_minimum)
filtered_test_dataset = test_dataset.filter(contains_minimum)

Filter:   0%|          | 0/9797 [00:00<?, ? examples/s]

Filter:   0%|          | 0/909 [00:00<?, ? examples/s]

In [ ]:
import random
def inject_memory_leak(code):
    """
    Injects memory leaks into the C++ program by inserting statements in plausible locations.
    """
    injections = [
        'char* leaked_string = (char*)malloc(50 * sizeof(char)); // Memory leak: not freed',
    ]
    # Inject a random memory leak after the first '{'
    modified_code = re.sub(r'(\{)', r'\1\n    ' + random.choice(injections), code, count=1)
    return modified_code

In [ ]:
def augment_example(example):
    """
    Modify an example by injecting a memory leak into the code column.
    """
    example['code'] = inject_memory_leak(example['code'])
    return example

# Apply the augmentation to the filtered datasets
augmented_train_dataset = filtered_train_dataset.map(augment_example)
augmented_test_dataset = filtered_test_dataset.map(augment_example)

Map:   0%|          | 0/1537 [00:00<?, ? examples/s]

Map:   0%|          | 0/138 [00:00<?, ? examples/s]

In [ ]:
#Replace Modified Examples in the Original Datasets
def replace_examples(original_dataset, augmented_dataset):
    """
    Replace examples in the original dataset with the augmented examples.
    """
    # Convert the augmented dataset to a dictionary for faster lookup
    augmented_dict = {ex['text']: ex for ex in augmented_dataset}

    # Replace matching examples in the original dataset
    def replace_if_matched(example):
        return augmented_dict.get(example['text'], example)

    return original_dataset.map(replace_if_matched)

# Replace the modified examples in the original datasets
final_train_dataset = replace_examples(dataset['train'], augmented_train_dataset)
final_test_dataset = replace_examples(dataset['test'], augmented_test_dataset)

Map:   0%|          | 0/9797 [00:00<?, ? examples/s]

Map:   0%|          | 0/909 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
#Save the Updated Datasets as CSV
def save_as_csv(dataset, filename):
    """
    Save a Hugging Face dataset as a CSV file.
    """
    # Convert to a pandas DataFrame and save
    df = pd.DataFrame(dataset)
    df.to_csv(filename, index=False)

save_as_csv(final_train_dataset, "augmented_train_dataset.csv")
save_as_csv(final_test_dataset, "augmented_test_dataset.csv")

print("Augmented train and test datasets saved as CSV files!")

Augmented train and test datasets saved as CSV files!


In [ ]:
trainer.save_model("/path/to/finetuned_phi3_code")
print("Fine-tuning complete. Model saved!")